In [1]:
!pip install -q -U google-generativeai

In [2]:
import pathlib
import textwrap
import pandas as pd

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown

from google.colab import drive
drive.mount('/content/gdrive')


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
api_key = ''
os.getenv(api_key)
genai.configure(api_key=api_key)

Loading data

In [4]:
df = pd.read_excel('/content/gdrive/MyDrive/CodingData/Euclid_Assignment/Gem_chapter_reduced_summary.xlsx', index_col = 'Unnamed: 0')
df.tail()

,book,chapter,text,chapter_num,chapter_summary
82,Harry Potter and the Goblet of Fire by,33,THE DEATH EATERS Voldemort looked away from Ha...,CHAPTER THIRTY-THREE,NaN
83,Harry Potter and the Goblet of Fire by,34,"PRIORI INCANTATEM Wormtail approached Harry, w...",CHAPTER THIRTY-FOUR,NaN
84,Harry Potter and the Goblet of Fire by,35,VERITASERUM Harry felt himself slam flat into ...,CHAPTER THIRTY-FIVE,NaN
85,Harry Potter and the Goblet of Fire by,36,THE PARTING OF THE WAYS Dumbledore stood up. H...,CHAPTER THIRTY-SIX,NaN
86,Harry Potter and the Goblet of Fire by,37,"THE BEGINNING When he looked back, even a mont...",CHAPTER THIRTY-SEVEN,NaN


In [5]:
df.fillna(0, inplace=True)
df.tail()

,book,chapter,text,chapter_num,chapter_summary
82,Harry Potter and the Goblet of Fire by,33,THE DEATH EATERS Voldemort looked away from Ha...,CHAPTER THIRTY-THREE,0
83,Harry Potter and the Goblet of Fire by,34,"PRIORI INCANTATEM Wormtail approached Harry, w...",CHAPTER THIRTY-FOUR,0
84,Harry Potter and the Goblet of Fire by,35,VERITASERUM Harry felt himself slam flat into ...,CHAPTER THIRTY-FIVE,0
85,Harry Potter and the Goblet of Fire by,36,THE PARTING OF THE WAYS Dumbledore stood up. H...,CHAPTER THIRTY-SIX,0
86,Harry Potter and the Goblet of Fire by,37,"THE BEGINNING When he looked back, even a mont...",CHAPTER THIRTY-SEVEN,0


Initializing Model

In [6]:
prompt = '''Summarize the below chunk of text, taken from a Harry Potter book, in strictly one sentence. \nKeep only the most relevant information, in the context of the whole book. \nText chunk: "<text>" \nSummarized output:'''


In [7]:
model = genai.GenerativeModel('gemini-pro')

In [39]:
text = df.text.iloc[0] + df.text.iloc[1] + df.text.iloc[2] #+ df.text.iloc[3] #+ df.text.iloc[4]
text = prompt.replace('<text>', text)

In [47]:
str(model.count_tokens(text)).split()[-1]

'15574'

In [41]:
resp = model.generate_content('Summarize: ' + text)

In [42]:
print(resp.text)

The Potters, a family of wizards, were killed by the evil wizard, Voldemort, when Harry Potter was a baby. Harry was left with his horrible Muggle relatives, the Dursleys, who treated him very badly. On his eleventh birthday, Harry receives a letter inviting him to attend Hogwarts School of Witchcraft and Wizardry. Despite the Dursleys' attempts to prevent him from going, Harry eventually makes his way to Hogwarts, where he begins to learn about his magical powers and his parents' past.


In [43]:
# on giving very large text, token around 29775, model was hallucinating
# hallucination started from 20k tokens, as whatever was given input was given as the output
# 10k token good response
# using 16k tokens per hit as its giving good response

In [45]:
class SummarizerGemini:
  def __init__(self, model): #, chapter_token_limit):
    self.model = model
    # self.chapter_token_limit = chapter_token_limit

  def get_token_count(self, text):
    token_count = self.model.count_tokens(text)
    return int(str(token_count).split()[-1])

  def get_model_output(self, text):
    print('Model input text: ', text)
    resp = model.generate_content(text)
    return resp.text

  def input_prompts(self, text, mode):
    chapter_prompt = '''Summarize the below chapter text, taken from a Harry Potter book, within two sentences. \nKeep only the most relevant information, in the context of the whole book. \nText chunk: "<text>" \nSummarized output:'''
    book_prompt = """Summarize the below chunk of chapter summaries taken from Harry Potter book in strictly below 3 lines, keeping only the most relevant information and theme, to get the essence of the whole book. \nChapter wise text chunk: "<text>" \nSummarized output:"""
    final_book_prompt = "Summarize the below summary of Harry Potter book capturing the primary setting, themes, context, characters development, challenges faced, significant events, revelation, conflicts and resolution in utmost detail in strictly 6 lines. \nText: '<text>' \nSummarized output:"

    assert mode in ['c', 'b', 'f'], "mode should be from ['c', 'b', 'f'], please enter a valid mode."
    if mode == 'c': return chapter_prompt.replace('<text>', text)
    if mode == 'b': return book_prompt.replace('<text>', text)
    if mode == 'f': return final_book_prompt.replace('<text>', text)

    return None

  def generate_chapter_summary(self, chapter_text):
    chapter_text = chapter_text.strip()
    chapter_token_length = self.get_token_count(chapter_text)

    if chapter_token_length>15000:
      chapter_text_list = chapter_text.split()
      length = len(chapter_text_list)
      print('Length of very large text: ', length)
      print('List of half words: ', chapter_text_list[:length//2])
      chapter_text_summary1 = self.get_model_output(self.input_prompts(' '.join(chapter_text_list[:length//2]), mode = 'c'))
      chapter_text_summary2 = self.get_model_output(self.input_prompts(' '.join(chapter_text_list[length//2:]), mode = 'c'))

      final_chapter_summary = self.get_model_output(self.input_prompts(chapter_text_summary1 + '\n' + chapter_text_summary2, mode = 'c'))
    else:
      final_chapter_summary = self.get_model_output(self.input_prompts(chapter_text, mode = 'c'))

    return final_chapter_summary

  def get_chapter_summary_df(self, df):
    file_save_location = '/content/gdrive/MyDrive/CodingData/Euclid_Assignment/Gem_chapter_reduced_summary1.xlsx'

    for idx, row in df.iterrows():
      if row.chapter_summary != 0:
        continue
      print('Row text: ',row.text)
      print('idx: ', idx)
      chap_summary = self.generate_chapter_summary(row.text)
      df.chapter_summary.iloc[idx] = chap_summary
      print(idx)
      print('Chapter: ', row.chapter)
      print('Summary: ', chap_summary)

      df.to_excel(file_save_location)

    return df

  def get_book_summary(self, df):
    books = []
    book_summary_list = []
    for book, book_df in df.groupby('book'):
      chapter_summary_text = '\n'.join(book_df.chapter_summary.to_list())
      book_summary = self.get_model_output(self.input_prompts(chapter_summary_text, mode = 'b'))

      books.append(book)
      book_summary_list.append(book_summary)

      print('Book: ', book)
      print('Summary: ', book_summary)

    data = {'book':books,
            'summary': book_summary_list}
    book_summary_df = pd.DataFrame(data)

    file_save_location = '/content/gdrive/MyDrive/CodingData/Euclid_Assignment/Gem_book_summary.xlsx'
    book_summary_df.to_excel(file_save_location)

    return book_summary_df

  def collective_summary(self, book_summary_df, book_order):
    book_summary_text = ''
    for book in book_order:
      book_summary = book_summary_df[book_summary_df['book'] == book]['summary'].iloc[0]
      book_summary_text += '\n' + book_summary
    final_summary = self.get_model_output(self.input_prompts(book_summary_text, mode = 'f'))
    return final_summary




In [46]:
obj = SummarizerGemini(model)

In [16]:
chapter_summary_df = obj.get_chapter_summary_df(df)

Row text:  O U R AT F L 0 V R I S H AND BLOTTS ife at the Burrow was as different as possible from life on Privet Drive. The Dursleys liked everything neat and ordered; the Weasleys' house burst with the strange and unexpected. Harry got a shock the first time he looked in the mirror over the kitchen mantelpiece and it shouted, "Tuck your shirt in, scruffy!" The ghoul in the attic howled and dropped pipes whenever he felt things were getting too quiet, and small explosions from Fred and George's bedroom were considered perfectly normal. What Harry found most unusual about life at Ron's, however, wasn't the talking mirror or the clanking ghoul: It was the fact that everybody there seemed to like him. Mrs. Weasley fussed over the state of his socks and tried to force him to eat fourth helpings at every meal. Mr. Weasley liked Harry to sit next to him at the dinner table so that he could bombard him with questions about life with Muggles, asking him to explain how things like plugs and th

<ipython-input-14-faf263598115>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.chapter_summary.iloc[idx] = chap_summary


20
Chapter:  4
Summary:  In this chapter, Harry is sent by the Weasleys to buy school supplies. While in the book shop, Gilderoy Lockhart reveals that he will be teaching at Hogwarts and Harry gets into a fight with Draco Malfoy.
Row text:  I V E THE WHOMPING WILLOW he end of the summer vacation came too quickly for Harry's liking. He was looking forward to getting back to Hogwarts, but his month at the Burrow had been the happiest of his life. It was difficult not to feel jealous of Ron when he thought of the Dursleys and the sort of welcome he could expect next time he turned up on Privet Drive. On their last evening, Mrs. Weasley conjured up a sumptuous dinner that included all of Harry's favorite things, ending with a mouthwatering treacle pudding. Fred and George rounded off the evening with a display of Filibuster fireworks; they fiIled the kitchen with red and blue stars that bounced from ceiling to wall for at least half an hour. Then it was time for a last mug of hot chocolate

In [21]:
book_summary_df = obj.get_book_summary(chapter_summary_df)

Model input text:  Summarize the below chunk of chapter summaries taken from Harry Potter book in strictly below 3 lines, keeping only the most relevant information and theme, to get the essence of the whole book. 
Chapter wise text chunk: "At the Dursleys, Harry's twelfth birthday is forgotten as they prepare for a dinner party with important guests. Harry feels isolated from his wizarding friends and deeply misses Hogwarts.
Dobby the house-elf warns Harry against returning to Hogwarts, claiming grave danger awaits him and offers to protect him. Despite Dobby's warning, Harry is determined to return to Hogwarts, leading to a conflict with his relatives and the Ministry of Magic, who discover his underage magic use.
Harry, Ron, and the twins rescue Harry from his awful home with the Dursleys. They have a joyful departure, but Mrs. Weasley is furious with them for using a flying car without permission. Despite this, Harry feels a sense of love and acceptance in the Weasley household.
In

In [23]:
for idx, row in book_summary_df.iterrows():
  print(row.book)
  print(row.summary)
  print('*'*100)

  HARRY POTTER AND  THE CHAMBER OF SECRETS      J. K. Rowling  
- Harry Potter is warned against returning to Hogwarts, but he disobeys and faces conflicts with his relatives and the Ministry of Magic.
- At Hogwarts, the Chamber of Secrets is opened, and a series of attacks on students begins. Harry, Ron, and Hermione investigate and uncover the legend of the Chamber and its connection to Salazar Slytherin's heir.
- Harry, Ron, and Hermione successfully defeat Voldemort's diary and save Ginny Weasley, unmasking Professor Lockhart as a fraud and leading to his dismissal.
****************************************************************************************************
Harry Potter  and  the Prisoner of Azkaban        
- Harry Potter, faced with adversity at his Muggle relatives', escapes to the wizarding world, learns about his past, and discovers dark secrets surrounding his parents' deaths.


- Harry faces challenges and dangers at Hogwarts, including encounters with dementors, bogg

In [26]:
book_order = df.book.unique()

In [47]:
print(obj.collective_summary(book_summary_df, book_order))

Model input text:  Summarize the below summary of Harry Potter book capturing the primary setting, themes, context, characters development, challenges faced, significant events, revelation, conflicts and resolution in utmost detail in strictly 6 lines. 
Text: '
- Harry Potter discovers he is a wizard and attends Hogwarts School of Witchcraft and Wizardry.


- Harry and his friends face challenges, including a three-headed dog, a troll, and a chess game, to retrieve the Sorcerer's Stone before Voldemort does.


- Harry defeats Lord Voldemort and saves the Sorcerer's Stone.
- Harry Potter is warned against returning to Hogwarts, but he disobeys and faces conflicts with his relatives and the Ministry of Magic.
- At Hogwarts, the Chamber of Secrets is opened, and a series of attacks on students begins. Harry, Ron, and Hermione investigate and uncover the legend of the Chamber and its connection to Salazar Slytherin's heir.
- Harry, Ron, and Hermione successfully defeat Voldemort's diary an